In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
import xlnet
import numpy as np
import tensorflow as tf
from tqdm import tqdm
import model_utils

In [3]:
import sentencepiece as spm
from prepro_utils import preprocess_text, encode_ids

sp_model = spm.SentencePieceProcessor()
sp_model.Load('alxlnet-base/sp10m.cased.v9.model')

True

In [4]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [5]:
from rules import normalized_chars
import random
import re
from unidecode import unidecode

laughing = {
    'huhu',
    'haha',
    'gagaga',
    'hihi',
    'wkawka',
    'wkwk',
    'kiki',
    'keke',
    'huehue',
    'hshs',
    'hoho',
    'hewhew',
    'uwu',
    'sksk',
    'ksks',
    'gituu',
    'gitu',
    'mmeeooww',
    'meow',
    'alhamdulillah',
    'muah',
    'mmuahh',
    'hehe',
    'salamramadhan',
    'happywomensday',
    'jahagaha',
    'ahakss',
    'ahksk'
}

def make_cleaning(s, c_dict):
    s = s.translate(c_dict)
    return s

def cleaning(string):
    """
    use by any transformer model before tokenization
    """
    string = unidecode(string)
    
    string = ' '.join(
        [make_cleaning(w, normalized_chars) for w in string.split()]
    )
    string = re.sub('\(dot\)', '.', string)
    string = (
        re.sub(re.findall(r'\<a(.*?)\>', string)[0], '', string)
        if (len(re.findall(r'\<a (.*?)\>', string)) > 0)
        and ('href' in re.findall(r'\<a (.*?)\>', string)[0])
        else string
    )
    string = re.sub(
        r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', ' ', string
    )
    
    chars = '.,/'
    for c in chars:
        string = string.replace(c, f' {c} ')
        
    string = re.sub(r'[ ]+', ' ', string).strip().split()
    string = [w for w in string if w[0] != '@']
    x = []
    for word in string:
        word = word.lower()
        if any([laugh in word for laugh in laughing]):
            if random.random() >= 0.5:
                x.append(word)
        else:
            x.append(word)
    string = [w.title() if w[0].isupper() else w for w in x]
    return ' '.join(string)

In [6]:
labels = """
1. severe toxic
2. obscene
3. identity attack
4. insult
5. threat
6. asian
7. atheist
8. bisexual
9. black
10. buddhist
11. christian
12. female
13. heterosexual
14. indian
15. homosexual, gay or lesbian
16. intellectual or learning disability
17. jewish
18. latino
19. male
20. muslim
21. other disability
22. other gender
23. other race or ethnicity
24. other religion
25. other sexual orientation
26. physical disability
27. psychiatric or mental illness
28. transgender
29. white
30. malay
31. chinese
"""
labels = [l.split('. ')[1].strip() for l in labels.split('\n') if len(l)]
labels

['severe toxic',
 'obscene',
 'identity attack',
 'insult',
 'threat',
 'asian',
 'atheist',
 'bisexual',
 'black',
 'buddhist',
 'christian',
 'female',
 'heterosexual',
 'indian',
 'homosexual, gay or lesbian',
 'intellectual or learning disability',
 'jewish',
 'latino',
 'male',
 'muslim',
 'other disability',
 'other gender',
 'other race or ethnicity',
 'other religion',
 'other sexual orientation',
 'physical disability',
 'psychiatric or mental illness',
 'transgender',
 'white',
 'malay',
 'chinese']

In [7]:
import glob

files = glob.glob('../toxicity/translated*')
files

['../toxicity/translated-1750000.json',
 '../toxicity/translated-1450000.json',
 '../toxicity/translated-700000.json',
 '../toxicity/translated-350000.json',
 '../toxicity/translated-600000.json',
 '../toxicity/translated-900000.json',
 '../toxicity/translated-1000000.json',
 '../toxicity/translated-1100000.json',
 '../toxicity/translated-550000.json',
 '../toxicity/translated-150000.json',
 '../toxicity/translated-500000.json',
 '../toxicity/translated-1500000.json',
 '../toxicity/translated-1150000.json',
 '../toxicity/translated-750000.json',
 '../toxicity/translated-850000.json',
 '../toxicity/translated-1650000.json',
 '../toxicity/translated-300000.json',
 '../toxicity/translated-650000.json',
 '../toxicity/translated-950000.json',
 '../toxicity/translated-250000.json',
 '../toxicity/translated-1600000.json',
 '../toxicity/translated-0.json',
 '../toxicity/translated-1550000.json',
 '../toxicity/translated-1800000.json',
 '../toxicity/translated-450000.json',
 '../toxicity/transl

In [8]:
import json

X, Y = [], []

for file in files:
    print(file)
    with open(file) as fopen:
        f = json.load(fopen)
        for row in f:
            if len(row[1]) == 29:
                X.append(row[0])
                Y.append(row[1] + [0, 0])
        
    
len(X)

../toxicity/translated-1750000.json
../toxicity/translated-1450000.json
../toxicity/translated-700000.json
../toxicity/translated-350000.json
../toxicity/translated-600000.json
../toxicity/translated-900000.json
../toxicity/translated-1000000.json
../toxicity/translated-1100000.json
../toxicity/translated-550000.json
../toxicity/translated-150000.json
../toxicity/translated-500000.json
../toxicity/translated-1500000.json
../toxicity/translated-1150000.json
../toxicity/translated-750000.json
../toxicity/translated-850000.json
../toxicity/translated-1650000.json
../toxicity/translated-300000.json
../toxicity/translated-650000.json
../toxicity/translated-950000.json
../toxicity/translated-250000.json
../toxicity/translated-1600000.json
../toxicity/translated-0.json
../toxicity/translated-1550000.json
../toxicity/translated-1800000.json
../toxicity/translated-450000.json
../toxicity/translated-50000.json
../toxicity/translated-1050000.json
../toxicity/translated-1200000.json
../toxicity/tr

1401054

In [9]:
rejected_labels = ['black', 'white', 'jewish', 'latino']
[labels.index(l) for l in rejected_labels]
labels = [l for l in labels if l not in rejected_labels]

In [10]:
ydf = pd.DataFrame(np.array(Y))
ydf = ydf.loc[(ydf[8] == 0) & (ydf[28] == 0) & (ydf[16] == 0) & (ydf[17] == 0)]
ydf = ydf.drop([8, 28, 16, 17], axis = 1)
ix = ydf.index.tolist()
Y = ydf.values.tolist()

In [11]:
X = [X[i] for i in ix]
len(X), len(Y)

(1361040, 1361040)

In [12]:
mapping = {'severe_toxic': 'severe toxic', 'identity_hate': 'identity attack',
          'toxic': 'severe toxic', 'melayu': 'malay', 'cina': 'chinese', 'india': 'indian'}

In [13]:
def generate_onehot(tags, depth = len(labels)):
    onehot = [0] * depth
    for tag in tags:
        onehot[labels.index(tag)] = 1
    return onehot

In [14]:
with open('../toxicity/kaum.json') as fopen:
    kaum = json.load(fopen)
    
for k, v in kaum.items():
    print(k, len(v))

melayu 84851
cina 43956
india 20208


In [15]:
with open('../toxicity/weak-learning-toxicity.json') as fopen:
    scores = json.load(fopen)
    
for k, v in scores.items():
    for no in range(len(v)):
        tags = []
        for l, v_ in v[no].items():
            if round(v_) == 1:
                tags.append(mapping.get(l, l))
        tags.append(mapping[k])
        Y.append(generate_onehot(tags))
        X.append(kaum[k][no])

In [16]:
from tqdm import tqdm

for i in tqdm(range(len(X))):
    X[i] = cleaning(X[i])

100%|██████████| 1510055/1510055 [04:46<00:00, 5279.91it/s] 


In [17]:
actual_t, actual_l = [], []

for i in tqdm(range(len(X))):
    if len(X[i]) > 2:
        actual_t.append(X[i])
        actual_l.append(Y[i])

100%|██████████| 1510055/1510055 [00:01<00:00, 1034178.33it/s]


In [18]:
from prepro_utils import preprocess_text, encode_ids

def tokenize_fn(text):
    text = preprocess_text(text, lower= False)
    return encode_ids(sp_model, text)

In [19]:
from tqdm import tqdm

SEG_ID_A   = 0
SEG_ID_B   = 1
SEG_ID_CLS = 2
SEG_ID_SEP = 3
SEG_ID_PAD = 4
MAX_SEQ_LENGTH = 150

special_symbols = {
    "<unk>"  : 0,
    "<s>"    : 1,
    "</s>"   : 2,
    "<cls>"  : 3,
    "<sep>"  : 4,
    "<pad>"  : 5,
    "<mask>" : 6,
    "<eod>"  : 7,
    "<eop>"  : 8,
}

VOCAB_SIZE = 32000
UNK_ID = special_symbols["<unk>"]
CLS_ID = special_symbols["<cls>"]
SEP_ID = special_symbols["<sep>"]
MASK_ID = special_symbols["<mask>"]
EOD_ID = special_symbols["<eod>"]

input_ids, input_masks, segment_ids = [], [], []

for text in tqdm(actual_t):
    tokens_a = tokenize_fn(text)
    if len(tokens_a) > MAX_SEQ_LENGTH - 2:
        tokens_a = tokens_a[:(MAX_SEQ_LENGTH - 2)]
        
    tokens = []
    segment_id = []
    for token in tokens_a:
        tokens.append(token)
        segment_id.append(SEG_ID_A)
    tokens.append(SEP_ID)
    segment_id.append(SEG_ID_A)
    tokens.append(CLS_ID)
    segment_id.append(SEG_ID_CLS)
    
    input_id = tokens
    input_mask = [0] * len(input_id)
    if len(input_id) < MAX_SEQ_LENGTH:
        delta_len = MAX_SEQ_LENGTH - len(input_id)
        input_id = [0] * delta_len + input_id
        input_mask = [1] * delta_len + input_mask
        segment_id = [SEG_ID_PAD] * delta_len + segment_id
    
    input_ids.append(input_id)
    input_masks.append(input_mask)
    segment_ids.append(segment_id)

100%|██████████| 1502328/1502328 [10:15<00:00, 2439.38it/s]


In [21]:
kwargs = dict(
      is_training=True,
      use_tpu=False,
      use_bfloat16=False,
      dropout=0.1,
      dropatt=0.1,
      init='normal',
      init_range=0.1,
      init_std=0.05,
      clamp_len=-1)

xlnet_parameters = xlnet.RunConfig(**kwargs)
xlnet_config = xlnet.XLNetConfig(json_path='alxlnet-base/config.json')

In [22]:
epoch = 10
batch_size = 50
warmup_proportion = 0.1
num_train_steps = int(len(actual_t) / batch_size * epoch)
num_warmup_steps = int(num_train_steps * warmup_proportion)
print(num_train_steps, num_warmup_steps)
learning_rate = 2e-5

training_parameters = dict(
      decay_method = 'poly',
      train_steps = num_train_steps,
      learning_rate = learning_rate,
      warmup_steps = num_warmup_steps,
      min_lr_ratio = 0.0,
      weight_decay = 0.00,
      adam_epsilon = 1e-8,
      num_core_per_host = 1,
      lr_layer_decay_rate = 1,
      use_tpu=False,
      use_bfloat16=False,
      dropout=0.0,
      dropatt=0.0,
      init='normal',
      init_range=0.1,
      init_std=0.05,
      clip = 1.0,
      clamp_len=-1,)

300465 30046


In [23]:
class Parameter:
    def __init__(self, decay_method, warmup_steps, weight_decay, adam_epsilon, 
                num_core_per_host, lr_layer_decay_rate, use_tpu, learning_rate, train_steps,
                min_lr_ratio, clip, **kwargs):
        self.decay_method = decay_method
        self.warmup_steps = warmup_steps
        self.weight_decay = weight_decay
        self.adam_epsilon = adam_epsilon
        self.num_core_per_host = num_core_per_host
        self.lr_layer_decay_rate = lr_layer_decay_rate
        self.use_tpu = use_tpu
        self.learning_rate = learning_rate
        self.train_steps = train_steps
        self.min_lr_ratio = min_lr_ratio
        self.clip = clip
        
training_parameters = Parameter(**training_parameters)

In [24]:
class Model:
    def __init__(
        self,
        dimension_output,
        learning_rate = 2e-5,
    ):
        self.X = tf.placeholder(tf.int32, [None, None])
        self.segment_ids = tf.placeholder(tf.int32, [None, None])
        self.input_masks = tf.placeholder(tf.float32, [None, None])
        self.Y = tf.placeholder(tf.float32, [None, dimension_output])
        
        xlnet_model = xlnet.XLNetModel(
            xlnet_config=xlnet_config,
            run_config=xlnet_parameters,
            input_ids=tf.transpose(self.X, [1, 0]),
            seg_ids=tf.transpose(self.segment_ids, [1, 0]),
            input_mask=tf.transpose(self.input_masks, [1, 0]))
        
        output_layer = xlnet_model.get_sequence_output()
        output_layer = tf.transpose(output_layer, [1, 0, 2])
        
        self.logits_seq = tf.layers.dense(output_layer, dimension_output)
        self.logits_seq = tf.identity(self.logits_seq, name = 'logits_seq')
        self.logits = self.logits_seq[:, 0]
        self.logits = tf.identity(self.logits, name = 'logits')
        
        self.cost = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(
                logits = self.logits, labels = self.Y
            )
        )
        
        self.optimizer, self.learning_rate, _ = model_utils.get_train_op(training_parameters, self.cost)
        
        correct_prediction = tf.equal(tf.round(tf.nn.sigmoid(self.logits)), tf.round(self.Y))
        all_labels_true = tf.reduce_min(tf.cast(correct_prediction, tf.float32), 1)
        self.accuracy = tf.reduce_mean(all_labels_true)

In [25]:
dimension_output = len(labels)

tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(
    dimension_output,
    learning_rate
)

sess.run(tf.global_variables_initializer())




INFO:tensorflow:memory input None
INFO:tensorflow:Use float type <dtype: 'float32'>

Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Please use `layer.__call__` method instead.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





In [26]:
import collections
import re

def get_assignment_map_from_checkpoint(tvars, init_checkpoint):
    """Compute the union of the current variables and checkpoint variables."""
    assignment_map = {}
    initialized_variable_names = {}

    name_to_variable = collections.OrderedDict()
    for var in tvars:
        name = var.name
        m = re.match('^(.*):\\d+$', name)
        if m is not None:
            name = m.group(1)
        name_to_variable[name] = var

    init_vars = tf.train.list_variables(init_checkpoint)

    assignment_map = collections.OrderedDict()
    for x in init_vars:
        (name, var) = (x[0], x[1])
        if name not in name_to_variable:
            continue
        assignment_map[name] = name_to_variable[name]
        initialized_variable_names[name] = 1
        initialized_variable_names[name + ':0'] = 1

    return (assignment_map, initialized_variable_names)

In [27]:
tvars = tf.trainable_variables()
checkpoint = 'alxlnet-base/model.ckpt'
assignment_map, initialized_variable_names = get_assignment_map_from_checkpoint(tvars, 
                                                                                checkpoint)

In [28]:
saver = tf.train.Saver(var_list = assignment_map)
saver.restore(sess, checkpoint)

INFO:tensorflow:Restoring parameters from alxlnet-base/model.ckpt


In [29]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_masks, test_masks, train_segments, test_segments, train_Y, test_Y = train_test_split(
    input_ids, input_masks, segment_ids, actual_l, test_size = 0.2
)

In [30]:
pad_sequences = tf.keras.preprocessing.sequence.pad_sequences

i = 0
index = 4
batch_x = train_X[i : index]
batch_y = train_Y[i : index]
batch_masks = train_masks[i : index]
batch_segments = train_segments[i : index]

sess.run(
    [model.accuracy, model.cost],
    feed_dict = {
        model.X: batch_x,
        model.Y: batch_y,
        model.segment_ids: batch_segments,
        model.input_masks: batch_masks,
    },
)

[0.0, 0.9872991]

In [31]:
from tqdm import tqdm
import time


EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 1, 0, 0, 0

while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n' % (EPOCH))
        break

    train_acc, train_loss, test_acc, test_loss = [], [], [], []
    pbar = tqdm(
        range(0, len(train_X), batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        index = min(i + batch_size, len(train_X))
        batch_x = train_X[i : index]
        batch_y = train_Y[i : index]
        batch_masks = train_masks[i : index]
        batch_segments = train_segments[i : index]
        acc, cost, _ = sess.run(
            [model.accuracy, model.cost, model.optimizer],
            feed_dict = {
                model.X: batch_x,
                model.Y: batch_y,
                model.segment_ids: batch_segments,
                model.input_masks: batch_masks,
            },
        )
        train_loss.append(cost)
        train_acc.append(acc)
        pbar.set_postfix(cost = cost, accuracy = acc)
        
    pbar = tqdm(range(0, len(test_X), batch_size), desc = 'test minibatch loop')
    for i in pbar:
        index = min(i + batch_size, len(test_X))
        batch_x = test_X[i : index]
        batch_y = test_Y[i : index]
        batch_masks = test_masks[i : index]
        batch_segments = test_segments[i : index]
        acc, cost = sess.run(
            [model.accuracy, model.cost],
            feed_dict = {
                model.X: batch_x,
                model.Y: batch_y,
                model.segment_ids: batch_segments,
                model.input_masks: batch_masks,
            },
        )
        test_loss.append(cost)
        test_acc.append(acc)
        pbar.set_postfix(cost = cost, accuracy = acc)
        
    train_loss = np.mean(train_loss)
    train_acc = np.mean(train_acc)
    test_loss = np.mean(test_loss)
    test_acc = np.mean(test_acc)
        
    if test_acc > CURRENT_ACC:
        print(
            'epoch: %d, pass acc: %f, current acc: %f'
            % (EPOCH, CURRENT_ACC, test_acc)
        )
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1
        
    print('time taken:', time.time() - lasttime)
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (EPOCH, train_loss, train_acc, test_loss, test_acc)
    )
    EPOCH += 1

train minibatch loop:   0%|          | 0/24038 [00:00<?, ?it/s]

epoch: 0, pass acc: 0.000000, current acc: 0.910005
time taken: 21153.76011800766
epoch: 0, training loss: 0.042540, training acc: 0.867588, valid loss: 0.022529, valid acc: 0.910005



train minibatch loop:   0%|          | 0/24038 [00:00<?, ?it/s]

epoch: 1, pass acc: 0.910005, current acc: 0.916076
time taken: 21164.35333251953
epoch: 1, training loss: 0.021906, training acc: 0.912364, valid loss: 0.021220, valid acc: 0.916076



train minibatch loop:   0%|          | 0/24038 [00:00<?, ?it/s]

epoch: 2, pass acc: 0.916076, current acc: 0.917317
time taken: 21209.230727910995
epoch: 2, training loss: 0.020763, training acc: 0.918800, valid loss: 0.020846, valid acc: 0.917317



train minibatch loop:   0%|          | 0/24038 [00:00<?, ?it/s]

epoch: 3, pass acc: 0.917317, current acc: 0.918731
time taken: 21187.296753406525
epoch: 3, training loss: 0.020114, training acc: 0.923085, valid loss: 0.020673, valid acc: 0.918731



test minibatch loop: 100%|██████████| 6010/6010 [28:47<00:00,  3.48it/s, accuracy=0.812, cost=0.0209]

time taken: 21198.428645133972
epoch: 4, training loss: 0.019618, training acc: 0.926257, valid loss: 0.020746, valid acc: 0.918218

break epoch:5



In [32]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'alxlnet-base-toxicity/model.ckpt')

'alxlnet-base-toxicity/model.ckpt'

In [33]:
kwargs = dict(
      is_training=False,
      use_tpu=False,
      use_bfloat16=False,
      dropout=0.0,
      dropatt=0.0,
      init='normal',
      init_range=0.1,
      init_std=0.05,
      clamp_len=-1)

xlnet_parameters = xlnet.RunConfig(**kwargs)
xlnet_config = xlnet.XLNetConfig(json_path='alxlnet-base/config.json')

In [34]:
dimension_output = len(labels)
learning_rate = 2e-5

tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(
    dimension_output,
    learning_rate
)

sess.run(tf.global_variables_initializer())

INFO:tensorflow:memory input None
INFO:tensorflow:Use float type <dtype: 'float32'>


/home/husein/.local/lib/python3.6/site-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [35]:
saver = tf.train.Saver(tf.trainable_variables())
saver.restore(sess, 'alxlnet-base-toxicity/model.ckpt')

INFO:tensorflow:Restoring parameters from alxlnet-base-toxicity/model.ckpt


In [36]:
stack = []

pbar = tqdm(range(0, len(test_X), batch_size), desc = 'test minibatch loop')
for i in pbar:
    index = min(i + batch_size, len(test_X))
    batch_x = test_X[i : index]
    batch_y = test_Y[i : index]
    batch_masks = test_masks[i : index]
    batch_segments = test_segments[i : index]
    stack.append(sess.run(tf.nn.sigmoid(model.logits),
            feed_dict = {
                model.X: batch_x,
                model.segment_ids: batch_segments,
                model.input_masks: batch_masks
            },
    ))

test minibatch loop: 100%|██████████| 6010/6010 [1:58:40<00:00,  1.18s/it]  


In [37]:
from sklearn import metrics

print(metrics.classification_report(np.around(np.array(test_Y)),
                                    np.around(np.concatenate(stack,axis=0)),
                                    target_names=labels,
                                    digits=5))

                                     precision    recall  f1-score   support

                       severe toxic    0.81360   0.70005   0.75257      9795
                            obscene    0.55653   0.49517   0.52406      2694
                    identity attack    0.50581   0.34938   0.41329      1371
                             insult    0.68040   0.60330   0.63953     12672
                             threat    0.40360   0.35123   0.37560       447
                              asian    0.66192   0.49077   0.56364       379
                            atheist    0.87151   0.92308   0.89655       169
                           bisexual    0.87500   0.60870   0.71795        23
                           buddhist    0.60417   0.63043   0.61702        46
                          christian    0.88051   0.84715   0.86351      4619
                             female    0.86958   0.92578   0.89680      6979
                       heterosexual    0.75000   0.82051   0.78367       11

/home/husein/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/husein/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/husein/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/husein/.local/lib/python3.6/site-packages/

In [38]:
strings = ','.join(
    [
        n.name
        for n in tf.get_default_graph().as_graph_def().node
        if ('Variable' in n.op
        or 'Placeholder' in n.name
        or 'logits' in n.name
        or 'alphas' in n.name
        or 'self/Softmax' in n.name)
        and 'Adam' not in n.name
        and 'beta' not in n.name
        and 'global_step' not in n.name
    ]
)
strings.split(',')

['Placeholder',
 'Placeholder_1',
 'Placeholder_2',
 'Placeholder_3',
 'model/transformer/r_w_bias',
 'model/transformer/r_r_bias',
 'model/transformer/word_embedding/lookup_table',
 'model/transformer/word_embedding/lookup_table_2',
 'model/transformer/r_s_bias',
 'model/transformer/seg_embed',
 'model/transformer/layer_shared/rel_attn/q/kernel',
 'model/transformer/layer_shared/rel_attn/k/kernel',
 'model/transformer/layer_shared/rel_attn/v/kernel',
 'model/transformer/layer_shared/rel_attn/r/kernel',
 'model/transformer/layer_shared/rel_attn/o/kernel',
 'model/transformer/layer_shared/rel_attn/LayerNorm/gamma',
 'model/transformer/layer_shared/ff/layer_1/kernel',
 'model/transformer/layer_shared/ff/layer_1/bias',
 'model/transformer/layer_shared/ff/layer_2/kernel',
 'model/transformer/layer_shared/ff/layer_2/bias',
 'model/transformer/layer_shared/ff/LayerNorm/gamma',
 'dense/kernel',
 'dense/bias',
 'logits_seq',
 'logits']

In [39]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            'directory: %s' % model_dir
        )

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path

    absolute_model_dir = '/'.join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + '/frozen_model.pb'
    clear_devices = True
    with tf.Session(graph = tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(
            input_checkpoint + '.meta', clear_devices = clear_devices
        )
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names.split(','),
        )
        with tf.gfile.GFile(output_graph, 'wb') as f:
            f.write(output_graph_def.SerializeToString())
        print('%d ops in the final graph.' % len(output_graph_def.node))

In [40]:
freeze_graph('alxlnet-base-toxicity', strings)

INFO:tensorflow:Restoring parameters from alxlnet-base-toxicity/model.ckpt
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 21 variables.
INFO:tensorflow:Converted 21 variables to const ops.
7390 ops in the final graph.


In [41]:
import boto3

bucketName = 'huseinhouse-storage'
Key = 'alxlnet-base-toxicity/frozen_model.pb'
outPutname = "v34/toxicity/alxlnet-base-toxicity.pb"

s3 = boto3.client('s3')

s3.upload_file(Key,bucketName,outPutname)